Installing the requirements.

In [ ]:
!pip install numpy==1.19.3
!pip install mediapipe
!pip install opencv-python
!pip install matplotlib
!pip install natsorted
!pip install pandas
!pip install plotly

Setting up automatic reload for the scripts that will be written outside the jupyter notebook.

In [1]:
%load_ext autoreload
%autoreload 2

Importing all dependencies.

In [2]:
from base64 import b64encode
import os
import sys
import time
import cv2
import mediapipe as mp
import pathlib
import matplotlib.pyplot as plt
from natsort import natsorted
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import HTML
from typing import List # I don't think I need this!

# Custom imports
from pose_tracking_utils import *

2023-02-23 09:36:38.312919: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 09:36:38.424121: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/lucassoares/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-02-23 09:36:38.424137: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-23 09:36:39.037613: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

Setting up all the necessary variables.

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

VIDEO_PATH = "./uchimata_wall.mp4"

Now we create the pose tracking video to visualize the joints and connections.

In [ ]:
# For webcam input:
output_path = create_pose_tracking_video("./videos/uchimata_wall.mp4")
print(output_path)

Compress the video and visualize it on jupyter notebook.

In [ ]:
compressed_path = save_compressed_video(output_path)
# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Create the landmark 3D plot animation.

In [8]:
i=0
pose_coords = []
# For video input
VIDEO_PATH = "./videos/uchimata_wall.mp4"
cap = cv2.VideoCapture(VIDEO_PATH)
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      break
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    mp_drawing.plot_landmarks_and_save(
         results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS, plot_index=i)
    # Draw the pose annotation on the image.
    # image.flags.writeable = True
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # mp_drawing.draw_landmarks(
    #     image,
    #     results.pose_landmarks,
    #     mp_pose.POSE_CONNECTIONS,
    #     landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    
    # Flip the image horizontally for a selfie-view display.
    #cv2.imshow("Pose viz",cv2.flip(image, 1))
    if cv2.waitKey(1) & 0xFF == 27:
      break
    
    i+=1
    

cap.release()

Ignoring empty camera frame.


In [9]:
create_animation_from_png(folder=".")

/tmp/ipykernel_22753/2278354264.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(os.path.join(directory, png_file))
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1000, 1000) to (1008, 1008) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


The pose landmarks are this:

Another list of pose landmarks in world coordinates. Each landmark consists of the following:

x, y and z: Real-world 3D coordinates in meters with the origin at the center between hips.
visibility: Identical to that defined in the corresponding

[source](https://google.github.io/mediapipe/solutions/pose.html)

Create the reference plot against which we will set objectives to improve
the desired movements. Like for example I am using olympiam judokas
as my reference to improve my uchimata movement.

(*This function and the interactive widget are not finished*)

In [10]:
def create_reference_plot(reference_coords: List, body_part_index: int):
    """
    Creates a reference plot for the desired motion
    Args:
        reference_coords (list): List of landmark objects with the x,y,z 
        coordinates.
        body_part_index (integer): index of the desired body part to plot.
    """
    body_part_coords = [reference_coords[i].landmark[body_part_index] for i in range(len(reference_coords))]
    
    # rest of the function code goes here


left_foot_coords = create_reference_plot(pose_coords, 31)
right_foot_coords = create_reference_plot(pose_coords, 32)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import ipywidgets as widgets
from IPython.display import display

# Extract x, y, z coordinates from left and right foot
x_coords = [coord.x for coord in left_foot_coords]
y_coords = [coord.y for coord in left_foot_coords]
z_coords = [coord.z for coord in left_foot_coords]

x_coords_right = [coord.x for coord in right_foot_coords]
y_coords_right = [coord.y for coord in right_foot_coords]
z_coords_right = [coord.z for coord in right_foot_coords]

# Create a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Foot Movement Over Time')

def update(frame):
    ax.clear()
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Foot Movement Over Time')
    ax.scatter(x_coords[:frame], y_coords[:frame], z_coords[:frame])
    ax.scatter(x_coords_right[:frame], y_coords_right[:frame], z_coords_right[:frame])

# Create a slider for selecting the frame
frame_slider = widgets.IntSlider(value=0, min=0, max=len(x_coords), step=1, description='Frame')

# Define a function that updates the plot when the slider is changed
def on_value_change(change):
    frame = change['new']
    update(frame)

# Attach the function to the slider
frame_slider.observe(on_value_change, names='value')

# Display the slider and the plot
display(widgets.VBox([frame_slider, fig.canvas]))

Here we create a 3D plot animation for body parts (in this case for the feet).

In [ ]:
x_coords = [coord.x for coord in left_foot_coords]
y_coords = [coord.y for coord in left_foot_coords]
z_coords = [coord.z for coord in left_foot_coords]

x_coords_right = [coord.x for coord in right_foot_coords]
y_coords_right = [coord.y for coord in right_foot_coords]
z_coords_right = [coord.z for coord in right_foot_coords]

# Create a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Foot Movement Over Time')

def update(frame):
    ax.clear()
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Foot Movement Over Time')
    ax.scatter(x_coords[:frame], y_coords[:frame], z_coords[:frame])
    ax.scatter(x_coords_right[:frame], y_coords_right[:frame], z_coords_right[:frame])

ani = FuncAnimation(fig, update, frames=len(x_coords), interval=5)
ani.save('animation.mp4', writer='ffmpeg', fps=30)
#plt.show()

Nicer looking landdmark plot of specific moments of the video, using plotly.

In [ ]:

plot_landmarks(pose_coords[200].pose_landmarks,  mp_pose.POSE_CONNECTIONS)